<!--NAVIGATION-->
<[Week-04-01-Data-Preprocessing-01](Week-04-01-Data-Preprocessing-01.ipynb) | [Contents](Index.ipynb) | [Week-05-01-Data-Preprocessing-03](Week-05-01-Data-Preprocessing-03.ipynb)  >




# Data Preprocessing 02
## Aggregation and Grouping

An essential piece of analysis of large data is efficient summarization: computing aggregations like
- ``sum()``, 
- ``mean()``, 
- ``median()``, 
- ``min()``, and 
- ``max()``, 

in which a single number gives insight into the nature of a potentially large dataset.

In this section, we'll explore aggregations in Pandas, from simple operations on what we have seen on NumPy arrays, to more sophisticated operations based on the concept of a ``groupby``.

## Additional Python (About Lambda)

Simply put, a lambda function is just like any normal python function, except that it has no name when defining it, and it is contained in one line of code.

In [1]:
# Add 10 to argument a, and return the result:
x = lambda a : a + 10
print(x(5))

15


In [2]:
# Multiply argument a with argument b and return the result:
x = lambda a, b : a * b
print(x(5, 6))

30


In [3]:
# Summarize argument a, b, and c and return the result:
x = lambda a, b, c : a + b + c
print(x(5, 6, 2))

13


## Background:

- Coarse-grained versus fine-grain data.
    - Coarse-grained: data with larger sub-categories (daily, weekly, monthly)
    - Fine-grain: data with smaller categories (hourly, minutely, secondly)
- Coarse-grained & Fine-grain data in both numerical dan categorical data
    - Time-based data: 
        - How many bike trips in one day? 
        - How many bike trips in one hour?
    - Category-based data: 
        - How many male visitors are there in the shopping mall? 
        - How many people do the take-away in the lunch time in our restaurant?


For convenience, we'll use the ``display`` magic function:

In [4]:
# Call the necessary library
import matplotlib.pyplot as plt #import the matplotlib
%matplotlib inline 
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

In [5]:
# Create a class in Python
# A class can consist many functions to describe the behavior of an object

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [6]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [7]:
tips.describe(include = "all")

,total_bill,tip,sex,smoker,day,time,size
count,244.000000,244.000000,244,244,244,244,244.000000
unique,NaN,NaN,2,2,4,2,NaN
top,NaN,NaN,Male,No,Sat,Dinner,NaN
freq,NaN,NaN,157,151,87,176,NaN
mean,19.785943,2.998279,NaN,NaN,NaN,NaN,2.569672
std,8.902412,1.383638,NaN,NaN,NaN,NaN,0.951100
min,3.070000,1.000000,NaN,NaN,NaN,NaN,1.000000
25%,13.347500,2.000000,NaN,NaN,NaN,NaN,2.000000
50%,17.795000,2.900000,NaN,NaN,NaN,NaN,2.000000
75%,24.127500,3.562500,NaN,NaN,NaN,NaN,3.000000


In [8]:
tips.dropna().describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [9]:
tips['total_bill'].mean()

19.785942622950824

## Simple Aggregation in Pandas

Earlier, we explored some of the data aggregations available for NumPy arrays.
As with a one-dimensional NumPy array, for a Pandas ``Series`` the aggregates return a single value:

The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

These are all methods of ``DataFrame`` and ``Series`` objects.

To go deeper into the data, however, simple aggregates are often not enough.
The next level of data summarization is the ``groupby`` operation, which allows you to quickly and efficiently compute aggregates on subsets of data.

## GroupBy: Split, Apply, Combine

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called ``groupby`` operation.
The name "group by" comes from a command in the SQL database language, but it is perhaps more illuminative to think of it in the terms first coined by Hadley Wickham of Rstats fame: *split, apply, combine*.

### Split, apply, combine

A canonical example of this split-apply-combine operation, where the "apply" is a summation aggregation, is illustrated in this figure:

![](figures/[04-02]_Fig_04_01_split-apply-combine.png) 

This makes clear what the ``groupby`` accomplishes:

- The *split* step involves breaking up and grouping a ``DataFrame`` depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
- The *combine* step merges the results of these operations into an output array.



While this could certainly be done manually using some combination of the masking, aggregation, and merging commands covered earlier, an important realization is that *the intermediate splits do not need to be explicitly instantiated*. Rather, the ``GroupBy`` can (often) do this in a single pass over the data, updating the sum, mean, count, min, or other aggregate for each group along the way.
The power of the ``GroupBy`` is that it abstracts away these steps: the user need not think about *how* the computation is done under the hood, but rather thinks about the *operation as a whole*.

As a concrete example, let's take a look at using Pandas for the computation shown in this diagram.
We'll start by creating the input ``DataFrame``:

In [10]:
tips.groupby("sex")

Notice that what is returned is not a set of ``DataFrame``s, but a ``DataFrameGroupBy`` object.
This object is where the magic is: you can think of it as a special view of the ``DataFrame``, which is poised to dig into the groups but does no actual computation until the aggregation is applied.
This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is almost transparent to the user.

To produce a result, we can apply an aggregate to this ``DataFrameGroupBy`` object, which will perform the appropriate apply/combine steps to produce the desired result:

In [11]:
tips.groupby("sex").mean()

,total_bill,tip,size
sex,,,
Male,20.744076,3.089618,2.630573
Female,18.056897,2.833448,2.459770


The ``sum()`` method is just one possibility here; you can apply virtually any common Pandas or NumPy aggregation function, as well as virtually any valid ``DataFrame`` operation, as we will see in the following discussion.

### The GroupBy object

The ``GroupBy`` object is a very flexible abstraction.
In many ways, you can simply treat it as if it's a collection of ``DataFrame``s, and it does the difficult things under the hood. Let's see some examples using the Planets data.

Perhaps the most important operations made available by a ``GroupBy`` are *aggregate*, *filter*, *transform*, and *apply*.
Here is to introduce some of the other functionality that can be used with the basic ``GroupBy`` operation.

#### Column indexing

The ``GroupBy`` object supports column indexing in the same way as the ``DataFrame``, and returns a modified ``GroupBy`` object.
For example:

In [12]:
tips.groupby('sex')

In [13]:
tips.groupby('sex')['total_bill']

Here we've selected a particular ``Series`` group from the original ``DataFrame`` group by reference to its column name.
As with the ``GroupBy`` object, no computation is done until we call some aggregate on the object:

In [14]:
tips.groupby('sex')['total_bill'].median()

sex
Male      18.35
Female    16.40
Name: total_bill, dtype: float64

#### Iteration over groups

The ``GroupBy`` object supports direct iteration over the groups, returning each group as a ``Series`` or ``DataFrame``:

In [15]:
for (method, group) in tips.groupby('day'):
    print("{0:10s} shape={1}".format(method, group.shape))

Thur       shape=(62, 7)
Fri        shape=(19, 7)
Sat        shape=(87, 7)
Sun        shape=(76, 7)


This can be useful for doing certain things manually, though it is often much faster to use the built-in ``apply`` functionality, which we will discuss momentarily.

#### Dispatch methods

Through some Python class magic, any method not explicitly implemented by the ``GroupBy`` object will be passed through and called on the groups, whether they are ``DataFrame`` or ``Series`` objects.
For example, you can use the ``describe()`` method of ``DataFrame``s to perform a set of aggregations that describe each group in the data:

In [16]:
tips.groupby('day')['total_bill'].describe()

,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
Thur,62.0,17.682742,7.886170,7.51,12.4425,16.20,20.1550,43.11
Fri,19.0,17.151579,8.302660,5.75,12.0950,15.38,21.7500,40.17
Sat,87.0,20.441379,9.480419,3.07,13.9050,18.24,24.7400,50.81
Sun,76.0,21.410000,8.832122,7.25,14.9875,19.63,25.5975,48.17


Looking at this table helps us to better understand the data: for example, the vast majority of planets have been discovered by the Radial Velocity and Transit methods, though the latter only became common (due to new, more accurate telescopes) in the last decade.
The newest methods seem to be Transit Timing Variation and Orbital Brightness Modulation, which were not used to discover a new planet until 2011.

This is just one example of the utility of dispatch methods.
Notice that they are applied *to each individual group*, and the results are then combined within ``GroupBy`` and returned.
Again, any valid ``DataFrame``/``Series`` method can be used on the corresponding ``GroupBy`` object, which allows for some very flexible and powerful operations!

# Learning Check

1. What is the different between aggregation and grouping (using groupby)?
2. In Groupby, there are three steps. What are they?

### Aggregate, filter, transform, apply

The preceding discussion focused on aggregation for the combine operation, but there are more options available.
In particular, ``GroupBy`` objects have ``aggregate()``, ``filter()``, ``transform()``, and ``apply()`` methods that efficiently implement a variety of useful operations before combining the grouped data.

For the purpose of the following subsections, we'll use this `tips` ``DataFrame``:

#### Aggregation

We're now familiar with ``GroupBy`` aggregations with ``sum()``, ``median()``, and the like, but the ``aggregate()`` method allows for even more flexibility.
It can take a string, a function, or a list thereof, and compute all the aggregates at once.
Here is a quick example combining all these:

In [17]:
# aggregate the day and calculate the minimum, median, and maximum from all numerical attributes
tips.groupby('day').aggregate(['min', np.median, max])

total_bill                 tip               size           
            min median    max   min median    max  min median max
day                                                              
Thur       7.51  16.20  43.11  1.25  2.305   6.70    1      2   6
Fri        5.75  15.38  40.17  1.00  3.000   4.73    1      2   4
Sat        3.07  18.24  50.81  1.00  2.750  10.00    1      2   5
Sun        7.25  19.63  48.17  1.01  3.150   6.50    2      2   6

Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column:

In [18]:
#Finding the minimum total bill from the maximum size
tips.groupby('day').aggregate({'total_bill': 'min',
                             'size': 'max'})

,total_bill,size
day,,
Thur,7.51,6
Fri,5.75,4
Sat,3.07,5
Sun,7.25,6


#### Filtering

A filtering operation allows you to drop data based on the group properties.
For example, we might want to keep all groups in which the standard deviation is larger than some critical value:

In [19]:
def filter_func(x):
    return x['total_bill'].std() > 20

display('tips.head()', "tips.groupby('day').std()", "tips.groupby('day').filter(filter_func)")

tips.head()
   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4

tips.groupby('day').std()
      total_bill       tip      size
day                                 
Thur    7.886170  1.240223  1.066285
Fri     8.302660  1.019577  0.567131
Sat     9.480419  1.631014  0.819275
Sun     8.832122  1.234880  1.007341

tips.groupby('day').filter(filter_func)
Empty DataFrame
Columns: [total_bill, tip, sex, smoker, day, time, size]
Index: []

The filter function should return a Boolean value specifying whether the group passes the filtering. Here because group A does not have a standard deviation greater than 4, it is dropped from the result.

#### Transformation

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine.
For such a transformation, the output is the same shape as the input.
A common example is to center the data by subtracting the group-wise mean:

In [20]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [21]:
tips['total_bill'].mean()

19.785942622950824

In [22]:
tips['total_bill'].head()

0    16.99
1    10.34
2    21.01
3    23.68
4    24.59
Name: total_bill, dtype: float64

In [23]:
tips.groupby('day')['total_bill'].mean()

day
Thur    17.682742
Fri     17.151579
Sat     20.441379
Sun     21.410000
Name: total_bill, dtype: float64

In [24]:
#normalize the data with the mean of the respective days
tips.groupby('day')['total_bill'].transform(lambda x: x - x.mean()).head()

0    -4.42
1   -11.07
2    -0.40
3     2.27
4     3.18
Name: total_bill, dtype: float64

In [25]:
#normalize the data with the mean of the respective days for all numerical attributes
tips.groupby('day').transform(lambda x: x - x.mean()).head()
#16.99 - 21.41 = -4.42
#10.34 - 21.41 = -11.07
#21.01 - 21.42 = -0.4

,total_bill,tip,size
0,-4.42,-2.245132,-0.842105
1,-11.07,-1.595132,0.157895
2,-0.40,0.244868,0.157895
3,2.27,0.054868,-0.842105
4,3.18,0.354868,1.157895


#### The apply() method

The ``apply()`` method lets you apply an arbitrary function to the group results.
The function should take a ``DataFrame``, and return either a Pandas object (e.g., ``DataFrame``, ``Series``) or a scalar; the combine operation will be tailored to the type of output returned.

For example, here is an ``apply()`` that normalizes the first column by the sum of the second:

In [37]:
tips['tip_percent'] = 100 * tips['tip'] / tips['total_bill']

In [27]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percent
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041
4,24.59,3.61,Female,No,Sun,Dinner,4,14.680765


In [28]:
tips['data1'] = tips['tip_percent']

In [29]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['tip_percent'].mean()
    return x

display('tips.head()', "tips.groupby('day').apply(norm_by_data2).head()")

,total_bill,tip,sex,smoker,day,time,size,tip_percent,data1
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673,5.944673
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159,16.054159
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734,16.658734
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041,13.978041
4,24.59,3.61,Female,No,Sun,Dinner,4,14.680765,14.680765
,total_bill,tip,sex,smoker,day,time,size,tip_percent,data1
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673,0.356188
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159,0.961918
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734,0.998143
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041,0.837524


``apply()`` within a ``GroupBy`` is quite flexible: the only criterion is that the function takes a ``DataFrame`` and returns a Pandas object or scalar; what you do in the middle is up to you!

### Additional Aggregate (Numerical Data)

There are some additional syntax for the aggregation such as cut and qcut

In [30]:
tips.describe()

,total_bill,tip,size,tip_percent,data1
count,244.000000,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,16.080258,16.080258
std,8.902412,1.383638,0.951100,6.107220,6.107220
min,3.070000,1.000000,1.000000,3.563814,3.563814
25%,13.347500,2.000000,2.000000,12.912736,12.912736
50%,17.795000,2.900000,2.000000,15.476977,15.476977
75%,24.127500,3.562500,3.000000,19.147549,19.147549
max,50.810000,10.000000,6.000000,71.034483,71.034483


In [31]:
#we cut the numerical values into some ranges (0 - 29, 30 - 59)
bill = pd.cut(tips['total_bill'], [0, 30, 60])

In [32]:
bill.head(20)

0      (0, 30]
1      (0, 30]
2      (0, 30]
3      (0, 30]
4      (0, 30]
5      (0, 30]
6      (0, 30]
7      (0, 30]
8      (0, 30]
9      (0, 30]
10     (0, 30]
11    (30, 60]
12     (0, 30]
13     (0, 30]
14     (0, 30]
15     (0, 30]
16     (0, 30]
17     (0, 30]
18     (0, 30]
19     (0, 30]
Name: total_bill, dtype: category
Categories (2, interval[int64]): [(0, 30] < (30, 60]]

In [33]:
#we cut the numerical values into 3 equal bins
bill2 = pd.cut(tips['total_bill'], bins=3)

In [34]:
bill2.head(20)

0      (3.022, 18.983]
1      (3.022, 18.983]
2     (18.983, 34.897]
3     (18.983, 34.897]
4     (18.983, 34.897]
5     (18.983, 34.897]
6      (3.022, 18.983]
7     (18.983, 34.897]
8      (3.022, 18.983]
9      (3.022, 18.983]
10     (3.022, 18.983]
11     (34.897, 50.81]
12     (3.022, 18.983]
13     (3.022, 18.983]
14     (3.022, 18.983]
15    (18.983, 34.897]
16     (3.022, 18.983]
17     (3.022, 18.983]
18     (3.022, 18.983]
19    (18.983, 34.897]
Name: total_bill, dtype: category
Categories (3, interval[float64]): [(3.022, 18.983] < (18.983, 34.897] < (34.897, 50.81]]

In [35]:
# we qcut (quantile cut) the tip into two quantiles (based on median)
tip_q = pd.qcut(tips['tip'], 2)

In [36]:
tip_q

0      (0.999, 2.9]
1      (0.999, 2.9]
2       (2.9, 10.0]
3       (2.9, 10.0]
4       (2.9, 10.0]
           ...     
239     (2.9, 10.0]
240    (0.999, 2.9]
241    (0.999, 2.9]
242    (0.999, 2.9]
243     (2.9, 10.0]
Name: tip, Length: 244, dtype: category
Categories (2, interval[float64]): [(0.999, 2.9] < (2.9, 10.0]]

## Summary

This section discusses examples on the use of `groupby` function in Pandas. In particular, the `split-apply-combine` approach is briefly introduced to deal with some basic functions such as aggregate, filtering, transformation, and apply